# BEST CLASSIFIERS

Load all people's responses into a list

In [1]:
import mne
import numpy as np

data_path = ''

epochs=[]
for i in [804, 807, 813, 815, 817]:
    data_file = str(i)+'_1_PDDys_ODDBALL_Clean_curated'
    filename = data_path + data_file
    current_epochs = mne.read_epochs(filename + '.fif')

    epochs.append(current_epochs['Standard', 'Novel'])


This filename (804_1_PDDys_ODDBALL_Clean_curated.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
Reading 804_1_PDDys_ODDBALL_Clean_curated.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
192 matching events found
No baseline correction applied
192 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated


<ipython-input-1-844e3f68764c>:10: RuntimeWarning: This filename (804_1_PDDys_ODDBALL_Clean_curated.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  current_epochs = mne.read_epochs(filename + '.fif')


This filename (807_1_PDDys_ODDBALL_Clean_curated.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
Reading 807_1_PDDys_ODDBALL_Clean_curated.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
186 matching events found
No baseline correction applied
186 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
This filename (813_1_PDDys_ODDBALL_Clean_curated.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
Reading 813_1_PDDys_ODDBALL_Clean_curated.fif ...


<ipython-input-1-844e3f68764c>:10: RuntimeWarning: This filename (807_1_PDDys_ODDBALL_Clean_curated.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  current_epochs = mne.read_epochs(filename + '.fif')
<ipython-input-1-844e3f68764c>:10: RuntimeWarning: This filename (813_1_PDDys_ODDBALL_Clean_curated.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  current_epochs = mne.read_epochs(filename + '.fif')


    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
190 matching events found
No baseline correction applied
190 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
This filename (815_1_PDDys_ODDBALL_Clean_curated.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
Reading 815_1_PDDys_ODDBALL_Clean_curated.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
192 matching events found
No baseline correction applied
192 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
This filename (817_1_PDDys_ODDBALL_Clean_curated.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
Reading 817_1_PDDys_ODDBALL_

<ipython-input-1-844e3f68764c>:10: RuntimeWarning: This filename (815_1_PDDys_ODDBALL_Clean_curated.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  current_epochs = mne.read_epochs(filename + '.fif')
<ipython-input-1-844e3f68764c>:10: RuntimeWarning: This filename (817_1_PDDys_ODDBALL_Clean_curated.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  current_epochs = mne.read_epochs(filename + '.fif')


189 matching events found
No baseline correction applied
189 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated


## Functions to get Test Data and Train Data

In [2]:
######################################################
#####     Get ALL DATA and add it together ###########
######################################################
# 
# m = Person that enters the room
#
def get_test_data(m):
    data_test= epochs[m]._data
    labels_test= epochs[m].events[:,-1]
    return data_test, labels_test
    
def get_train_data(m):
    initialised=0
    for i in range(0,5):
        if i != m:
            if initialised==0:
                data_train=epochs[i]._data
                labels_train=epochs[i].events[:,-1]
                initialised=1
            else: 
                data_train = np.concatenate((data_train, epochs[i]._data), axis=0)
                labels_train = np.concatenate((labels_train, epochs[i].events[:,-1]), axis=0)
    data_test, labels_test = get_test_data(m)
    return data_train, labels_train, data_test, labels_test
            




In [3]:
## Test #
#print(
#    np.shape(data_train),
#    np.shape(data_test),
#    np.shape(labels_test),
#    np.shape(labels_train),
#    )

# Import ML

In [4]:
from sklearn.model_selection import GridSearchCV
from mne.decoding import Vectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.model_selection import train_test_split

# Try different test person and different Frequency Filters

In [5]:
def sweep_test_person(i, l_freq=0.1, h_freq=20):
    for epoch in epochs:
        epoch.filter(l_freq = l_freq, h_freq = h_freq)
        #epoch.apply_baseline((None, 0))

    print("TEST PERSON ", str(i))
        
    data_train, labels_train, data_test, labels_test = get_train_data(i)  
    print(np.shape(data_train), np.shape(labels_train))
    print(np.shape(data_test), np.shape(labels_test))
    
    data= np.concatenate((data_train, data_test), axis=0)
    labels= np.concatenate((labels_train, labels_test), axis=0)
    
    clf_opt = make_pipeline(Vectorizer(), StandardScaler(), GridSearchCV(svm.SVC(), {'kernel':('poly', 'rbf')}))#, 'poly', 'linear', 'precomputed'), 'C':[0.1,0.5,1,2,3,5,8,10,20], 'degree':[2,3,4,5], 'random_state':[0,1,40,68,200]}
    clf_opt.fit(data_train, labels_train)

    print("Performance Trainset BestOption: ", clf_opt.steps[-1])

    clf_opt.predict(data_train)
    print("Score on Trainset: ", clf_opt.score(data_train, labels_train))

    clf_opt.predict(data_test)
    print("Score on Testset: ", clf_opt.score(data_test, labels_test))

    #scores = cross_val_score(clf_opt, data, labels, cv=3)
    #print("CrossVal Single Score: ", scores)
    #print("Accuracy CrossVal: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
 
    return clf_opt.score(data_test, labels_test) #,scores



# RUN Frequency Sweep and SELECT Frequency

In [6]:
##
scores_freq=[]
freq_settings=[[0.1, 20], [20,40], [4,20]]
#freq_settings=[[10,20]]
for l_freq, h_freq in freq_settings:
    
    
    epochs=[]
    for i in [804, 807, 813, 815, 817]:
        data_file = str(i)+'_1_PDDys_ODDBALL_Clean_curated'
        filename = data_path + data_file
        current_epochs = mne.read_epochs(filename + '.fif')

        epochs.append(current_epochs['Standard', 'Novel'])
    
    print(l_freq, h_freq)
    score = sweep_test_person(0, l_freq=l_freq, h_freq=h_freq)
    scores_freq.append(score)

This filename (804_1_PDDys_ODDBALL_Clean_curated.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
Reading 804_1_PDDys_ODDBALL_Clean_curated.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
192 matching events found
No baseline correction applied
192 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
This filename (807_1_PDDys_ODDBALL_Clean_curated.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
Reading 807_1_PDDys_ODDBALL_Clean_curated.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available


<ipython-input-6-cf8f8bd13f03>:12: RuntimeWarning: This filename (804_1_PDDys_ODDBALL_Clean_curated.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  current_epochs = mne.read_epochs(filename + '.fif')
<ipython-input-6-cf8f8bd13f03>:12: RuntimeWarning: This filename (807_1_PDDys_ODDBALL_Clean_curated.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  current_epochs = mne.read_epochs(filename + '.fif')


186 matching events found
No baseline correction applied
186 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
This filename (813_1_PDDys_ODDBALL_Clean_curated.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
Reading 813_1_PDDys_ODDBALL_Clean_curated.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
190 matching events found
No baseline correction applied
190 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
This filename (815_1_PDDys_ODDBALL_Clean_curated.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
Reading 815_1_PDDys_ODDBALL_Clean_curated.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensati

<ipython-input-6-cf8f8bd13f03>:12: RuntimeWarning: This filename (813_1_PDDys_ODDBALL_Clean_curated.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  current_epochs = mne.read_epochs(filename + '.fif')
<ipython-input-6-cf8f8bd13f03>:12: RuntimeWarning: This filename (815_1_PDDys_ODDBALL_Clean_curated.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  current_epochs = mne.read_epochs(filename + '.fif')


192 matching events found
No baseline correction applied
192 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
This filename (817_1_PDDys_ODDBALL_Clean_curated.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
Reading 817_1_PDDys_ODDBALL_Clean_curated.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
189 matching events found
No baseline correction applied
189 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
0.1 20
Setting up band-pass filter from 0.1 - 20 Hz
l_trans_bandwidth chosen to be 0.1 Hz
h_trans_bandwidth chosen to be 5.0 Hz
Filter length of 16501 samples (33.002 sec) selected
filter_length (16501) is longer than the signal (301), distortion is likely. Reduce filter length or filter a longer signal.


...

Setting up band-pass filter from 4 - 20 Hz
l_trans_bandwidth chosen to be 2.0 Hz
h_trans_bandwidth chosen to be 5.0 Hz
Filter length of 825 samples (1.650 sec) selected
filter_length (825) is longer than the signal (301), distortion is likely. Reduce filter length or filter a longer signal.


<ipython-input-5-cbb95587c355>:3: RuntimeWarning: filter_length (825) is longer than the signal (301), distortion is likely. Reduce filter length or filter a longer signal.
  epoch.filter(l_freq = l_freq, h_freq = h_freq)


Setting up band-pass filter from 4 - 20 Hz
l_trans_bandwidth chosen to be 2.0 Hz
h_trans_bandwidth chosen to be 5.0 Hz
Filter length of 825 samples (1.650 sec) selected
filter_length (825) is longer than the signal (301), distortion is likely. Reduce filter length or filter a longer signal.


<ipython-input-5-cbb95587c355>:3: RuntimeWarning: filter_length (825) is longer than the signal (301), distortion is likely. Reduce filter length or filter a longer signal.
  epoch.filter(l_freq = l_freq, h_freq = h_freq)


TEST PERSON  0
(640, 60, 301) (640,)
(162, 60, 301) (162,)
Performance Trainset BestOption:  ('gridsearchcv', GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'kernel': ('poly', 'rbf')}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=0))
Score on Trainset:  0.875
Score on Testset:  0.8209876543209876


## RESULTS FOR DIFFERNET FREQUENCY SETTINGS

In [7]:
print( freq_settings, scores_freq)

[0.8333333333333334, 0.8209876543209876, 0.8209876543209876]

#Choose the best frequency filter by hand: 

In [8]:
good_l_freq = 0.1
good_h_freq = 20

# Try a different test Person

In [9]:
score_people=[]
for i in range(0,5):
    score = sweep_test_person(i, l_freq= good_l_freq, h_freq=good_h_freq)
    score_people.append(score)

Setting up band-pass filter from 0.1 - 20 Hz
l_trans_bandwidth chosen to be 0.1 Hz
h_trans_bandwidth chosen to be 5.0 Hz
Filter length of 16501 samples (33.002 sec) selected
filter_length (16501) is longer than the signal (301), distortion is likely. Reduce filter length or filter a longer signal.


<ipython-input-5-cbb95587c355>:3: RuntimeWarning: filter_length (16501) is longer than the signal (301), distortion is likely. Reduce filter length or filter a longer signal.
  epoch.filter(l_freq = l_freq, h_freq = h_freq)


Setting up band-pass filter from 0.1 - 20 Hz
l_trans_bandwidth chosen to be 0.1 Hz
h_trans_bandwidth chosen to be 5.0 Hz
Filter length of 16501 samples (33.002 sec) selected
filter_length (16501) is longer than the signal (301), distortion is likely. Reduce filter length or filter a longer signal.


<ipython-input-5-cbb95587c355>:3: RuntimeWarning: filter_length (16501) is longer than the signal (301), distortion is likely. Reduce filter length or filter a longer signal.
  epoch.filter(l_freq = l_freq, h_freq = h_freq)


Setting up band-pass filter from 0.1 - 20 Hz
l_trans_bandwidth chosen to be 0.1 Hz
h_trans_bandwidth chosen to be 5.0 Hz
Filter length of 16501 samples (33.002 sec) selected
filter_length (16501) is longer than the signal (301), distortion is likely. Reduce filter length or filter a longer signal.


...

Setting up band-pass filter from 0.1 - 20 Hz
l_trans_bandwidth chosen to be 0.1 Hz
h_trans_bandwidth chosen to be 5.0 Hz
Filter length of 16501 samples (33.002 sec) selected
filter_length (16501) is longer than the signal (301), distortion is likely. Reduce filter length or filter a longer signal.


<ipython-input-5-cbb95587c355>:3: RuntimeWarning: filter_length (16501) is longer than the signal (301), distortion is likely. Reduce filter length or filter a longer signal.
  epoch.filter(l_freq = l_freq, h_freq = h_freq)


Setting up band-pass filter from 0.1 - 20 Hz
l_trans_bandwidth chosen to be 0.1 Hz
h_trans_bandwidth chosen to be 5.0 Hz
Filter length of 16501 samples (33.002 sec) selected
filter_length (16501) is longer than the signal (301), distortion is likely. Reduce filter length or filter a longer signal.


<ipython-input-5-cbb95587c355>:3: RuntimeWarning: filter_length (16501) is longer than the signal (301), distortion is likely. Reduce filter length or filter a longer signal.
  epoch.filter(l_freq = l_freq, h_freq = h_freq)


TEST PERSON  4
(642, 60, 301) (642,)
(160, 60, 301) (160,)
Performance Trainset BestOption:  ('gridsearchcv', GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'kernel': ('poly', 'rbf')}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=0))
Score on Trainset:  0.8738317757009346
Score on Testset:  0.8125


## Plot

In [14]:
from matplotlib import pyplot as plt
%matplotlib tk
plt.bar(np.arange(0,5), score_people, label='BEST_Classifier (one and only), l_freq='+str(good_l_freq)+'h_freq='+str(good_h_freq))
plt.ylim(0.75, 0.85)
plt.legend()
plt.xlabel("Who is the test person?")
plt.ylabel("Score on the given test person")
plt.show()